<a href="https://colab.research.google.com/github/jkf87/autogen-handson/blob/main/autogen%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%B4%EC%84%9C_%EC%B6%9C%EB%A0%A5_%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0%EB%A5%BC_%ED%8A%9C%EB%8B%9D%ED%95%98%EB%8A%94_%EB%B0%A9%EB%B2%95oai_chatgpt_gpt4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

저작권 (c) Microsoft Corporation. 모든 권리 보유.

MIT 라이선스에 따라 라이센스가 부여되었습니다.

# AutoGen을 사용하여 ChatGPT 튜닝하기

AutoGen은 대규모 언어 모델 튜닝을 위한 비용 효율적인 하이퍼파라미터 최적화 기법 [EcoOptiGen](https://arxiv.org/abs/2303.04673)을 제공합니다. 연구에 따르면 하이퍼파라미터를 튜닝하면 LLM의 유용성을 크게 향상시킬 수 있습니다.
이 기능에 대한 설명서는 [여기](/docs/Use-Cases/AutoGen#enhanced-inference)에서 확인할 수 있습니다.

이 노트북에서는 수학 문제 해결을 위해 OpenAI ChatGPT(GPT-3.5와 GPT-4 모두) 모델을 튜닝합니다. 우리는 [the MATH 벤치마크](https://crfm.stanford.edu/helm/latest/?group=math_chain_of_thought)를 사용하여 연쇄적 추론 스타일의 경쟁 수학 문제에서 수학 문제 해결력을 측정합니다.

관련 링크: 이 실험을 기반으로 한 [블로그 포스트](https://microsoft.github.io/autogen/blog/2023/04/21/LLM-tuning-math).

## 요구 사항

자동 생성에는 `Python>=3.8`이 필요합니다. 이 노트북 예제를 실행하려면 [blendsearch] 옵션으로 설치하세요:
```bash
pip install "pyautogen[blendsearch]"
```

In [ ]:
%pip install "pyautogen[blendsearch]" datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

AutoGen has provided an API for hyperparameter optimization of OpenAI ChatGPT models: `autogen.ChatCompletion.tune` and to make a request with the tuned config: `autogen.ChatCompletion.create`. First, we import autogen:

AutoGen은 OpenAI ChatGPT 모델의 하이퍼파라미터 최적화를 위한 API를 제공합니다: `autogen.ChatCompletion.tune` 을 호출하고 튜닝된 설정으로 요청할 수 있습니다: `autogen.ChatCompletion.create`를 제공합니다. 먼저 오토젠을 가져옵니다:


In [ ]:
import autogen

### Set your API Endpoint

The [`config_list_openai_aoai`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_openai_aoai) function tries to create a list of  Azure OpenAI endpoints and OpenAI endpoints. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure OpenAI API key + base.


### API 엔드포인트 설정

`config_list_openai_aoai`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_openai_aoai) 함수는 Azure OpenAI 엔드포인트 및 OpenAI 엔드포인트 목록을 생성하려고 시도합니다. 이 함수는 API 키와 API 베이스가 해당 환경 변수 또는 로컬 txt 파일에 저장되어 있다고 가정합니다:

- OpenAI API 키: os.environ["OPENAI_API_KEY"] 또는 `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API 키: os.environ["AZURE_OPENAI_API_KEY"] 또는 `aoai_api_key_file="key_aoai.txt"`. 한 줄에 하나씩 여러 키를 저장할 수 있습니다.
- Azure OpenAI API 베이스: os.environ["AZURE_OPENAI_API_BASE"] 또는 `aoai_api_base_file="base_aoai.txt"`. 한 줄에 하나씩 여러 개의 베이스를 저장할 수 있습니다.

OpenAI API 키만 있거나 Azure OpenAI API 키 + 베이스만 있어도 괜찮습니다.

In [ ]:
config_list = autogen.config_list_openai_aoai()

The config list looks like the following:
```python
config_list = [
    {'api_key': '<your OpenAI API key here>'},  # only if OpenAI API key is found
    {
        'api_key': '<your first Azure OpenAI API key here>',
        'api_base': '<your first Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },  # only if the at least one Azure OpenAI API key is found
    {
        'api_key': '<your second Azure OpenAI API key here>',
        'api_base': '<your second Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },  # only if the second Azure OpenAI API key is found
]
```

You can directly override it if the above function returns an empty list, i.e., it doesn't find the keys in the specified locations.

#API 엔드포인트 설정 코드 변경
- 기존 config_list 방식이 번거로운 분들을 위해 설정 셀 추가
- `!pip install python-dotenv`

```
import json
from dotenv import find_dotenv, load_dotenv

env_var = [
    {
        'model': 'gpt-4',
        'api_key': "sk-"
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': "sk-"
    }
]
```

In [ ]:
!pip install python-dotenv

In [ ]:
import json
from dotenv import find_dotenv, load_dotenv


env_var = [
    {
        'model': 'gpt-3.5-turbo',
        'api_key': "sk-"
    }

]
config_list = env_var

# 이 부분 아래 코드로 대체함
## Load dataset

We load the competition_math dataset. The dataset contains 201 "Level 2" Algebra examples. We use a random sample of 20 examples for tuning the generation hyperparameters and the remaining for evaluation.

## 데이터 세트 로드

competition_math 데이터 세트를 로드합니다. 이 데이터 세트에는 201개의 "레벨 2" 대수 예제가 포함되어 있습니다. 생성 하이퍼파라미터를 튜닝하기 위해 20개의 예제 중 무작위 샘플을 사용하고 나머지는 평가에 사용합니다.

In [ ]:
from datasets import list_datasets

# Get the list of available datasets
available_datasets = list_datasets()

# Print some of the datasets
print(available_datasets[:10])

<ipython-input-23-53ea4887b91e>:4: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  available_datasets = list_datasets()


['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc', 'afrikaans_ner_corpus', 'ag_news', 'ai2_arc', 'air_dialogue', 'ajgt_twitter_ar', 'allegro_reviews']


In [ ]:
import datasets

seed = 87
data = datasets.load_dataset("competition_math")
train_data = data["train"].shuffle(seed=seed)
test_data = data["test"].shuffle(seed=seed)
n_tune_data = 20
tune_data = [
    {
        "problem": train_data[x]["problem"],
        "solution": train_data[x]["solution"],
    }
    for x in range(len(train_data)) if train_data[x]["level"] == "Level 2" and train_data[x]["type"] == "Algebra"
][:n_tune_data]
test_data = [
    {
        "problem": test_data[x]["problem"],
        "solution": test_data[x]["solution"],
    }
    for x in range(len(test_data)) if test_data[x]["level"] == "Level 2" and test_data[x]["type"] == "Algebra"
]
print(len(tune_data), len(test_data))


# [대체코드]데이터셋에 맞춰 코드를 수정함
- 심리상담 데이터셋[웰니스 데이터셋](https://github.com/kairess/mental-health-chatbot/blob/master/wellness_dataset_original.csv)
- 매칭이 되지 않는 빈칸은 전처리함
- 랜덤 샘플 20개를 돌림

In [ ]:
import pandas as pd
def preprocess_wellness_data(file_path, seed=95, n_tune_samples=20):
    # Load the data from the given file path
    data = pd.read_csv(file_path)

    # 1. Remove NaN values
    data_cleaned = data.dropna(subset=['유저', '챗봇'])

    # 2. Shuffle the cleaned data with the provided seed
    data_shuffled = data_cleaned.sample(frac=1, random_state=seed).reset_index(drop=True)

    # 3. Select top n_tune_samples for tuning
    tune_data = data_shuffled.head(n_tune_samples).to_dict(orient='records')

    # 4. Rest for testing
    test_data = data_shuffled.tail(len(data_shuffled) - n_tune_samples).to_dict(orient='records')

    return tune_data, test_data

# Use the function to preprocess the data
tune_data, test_data = preprocess_wellness_data("/content/wellness_dataset_original.csv")

len(tune_data), len(test_data)

seed=95

Check a tuning example:

In [ ]:
print(tune_data[0]["유저"])

예민해서 잠을 설칠 때도 있구요.


Here is one example of the canonical solution:

In [ ]:
print(test_data[0]["챗봇"])

도망치는 것도 좋은 방법이에요. 때로는 피하는 것에도 용기가 필요하죠.


## Define Success Metric

Before we start tuning, we need to define the success metric we want to optimize. For each math task, we use voting to select a response with the most common answers out of all the generated responses. If it has an equivalent answer to the canonical solution, we consider the task as successfully solved. Then we can optimize the mean success rate of a collection of tasks.

## 성공 지표 정의

튜닝을 시작하기 전에 최적화하려는 성공 지표를 정의해야 합니다. 각 수학 과제에 대해 투표를 사용하여 생성된 모든 답변 중에서 가장 일반적인 답변을 가진 답변을 선택합니다. 정답과 동등한 답이 있는 경우, 해당 과제를 성공적으로 푼 것으로 간주합니다. 그러면 과제 모음의 평균 성공률을 최적화할 수 있습니다.

In [ ]:
from autogen.math_utils import eval_math_responses

## Use the tuning data to find a good configuration


For (local) reproducibility and cost efficiency, we cache responses from OpenAI with a controllable seed.

In [ ]:
autogen.ChatCompletion.set_cache(seed)

이렇게 하면 ".cache/{seed}"에 디스크 캐시가 생성됩니다. 캐시_경로_루트`를 `set_cache()`에서 ".cache"에서 다른 경로로 변경할 수 있습니다. 다른 시드에 대한 캐시는 별도로 저장됩니다.

### 튜닝 수행

튜닝은 최적화 예산에 따라 완료하는 데 시간이 걸릴 수 있습니다. 튜닝은 지정된 최적화 예산 내에서 수행됩니다.

* '추론_예산'은 벤치마크에서 인스턴스당 목표 평균 추론 예산입니다. 예를 들어 0.004는 목표 추론 예산이 0.004달러임을 의미하며, gpt-3.5 터보 모델을 사용하는 경우 2000토큰(입력 + 출력 합산)으로 해석됩니다.
* 'optimization_budget'은 튜닝을 수행하는 데 허용되는 총 예산입니다. 예를 들어 1은 총 1달러가 허용됨을 의미하며, 이는 gpt-3.5 터보 모델의 경우 50만 토큰을 의미합니다.
* 'num_sumples'는 시도할 수 있는 다양한 하이퍼파라미터 구성의 개수입니다. 튜닝은 num_samples 시도 후 또는 최적화_예산이 소비된 후 둘 중 먼저 발생한 후 중지됩니다. -1은 시도 횟수에 제한이 없으며 실제 횟수는 `optimization_budget`에 의해 결정됨을 의미합니다.

사용자는 튜닝 데이터, 최적화 지표, 최적화 모드, 평가 기능, 검색 공간 등을 지정할 수 있습니다. 기본 검색 공간은

```python
default_search_space = {
    "model": tune.choice([
        "gpt-3.5-turbo",
        "gpt-4",
    ]),
    "temperature_or_top_p": tune.choice(
        [
            {"temperature": tune.uniform(0, 2)},
            {"top_p": tune.uniform(0, 1)},
        ]
    ),
    "max_tokens": tune.lograndint(50, 1000),
    "n": tune.randint(1, 100),
    "prompt": "{프롬프트}",
}
```

기본 검색 공간은 사용자의 입력으로 재정의할 수 있습니다.
예를 들어 다음 코드는 고정 프롬프트 템플릿을 지정합니다. 사용자 입력에 나타나지 않는 하이퍼파라미터의 경우 기본 검색 공간이 사용됩니다.

This will create a disk cache in ".cache/{seed}". You can change `cache_path_root` from ".cache" to a different path in `set_cache()`. The cache for different seeds are stored separately.

### Perform tuning

The tuning will take a while to finish, depending on the optimization budget. The tuning will be performed under the specified optimization budgets.

* `inference_budget` is the target average inference budget per instance in the benchmark. For example, 0.004 means the target inference budget is 0.004 dollars, which translates to 2000 tokens (input + output combined) if the gpt-3.5-turbo model is used.
* `optimization_budget` is the total budget allowed to perform the tuning. For example, 1 means 1 dollars are allowed in total, which translates to 500K tokens for the gpt-3.5-turbo model.
* `num_sumples` is the number of different hyperparameter configurations which is allowed to try. The tuning will stop after either num_samples trials or after optimization_budget dollars spent, whichever happens first. -1 means no hard restriction in the number of trials and the actual number is decided by `optimization_budget`.

Users can specify tuning data, optimization metric, optimization mode, evaluation function, search spaces etc.. The default search space is:

```python
default_search_space = {
    "model": tune.choice([
        "gpt-3.5-turbo",
        "gpt-4",
    ]),
    "temperature_or_top_p": tune.choice(
        [
            {"temperature": tune.uniform(0, 2)},
            {"top_p": tune.uniform(0, 1)},
        ]
    ),
    "max_tokens": tune.lograndint(50, 1000),
    "n": tune.randint(1, 100),
    "prompt": "{prompt}",
}
```

The default search space can be overridden by users' input.
For example, the following code specifies a fixed prompt template. For hyperparameters which don't appear in users' input, the default search space will be used.

In [ ]:
import logging

prompts = ["{유저} Please listen to the concern carefully. Provide a thoughtful and empathetic response. Frame your answer within \\boxed{{}}."]
config, analysis = autogen.ChatCompletion.tune(
    data=tune_data,  # 튜닝을 위한 데이터
    metric="success_vote",  # 최적화할 메트릭
    mode="max",  # 최적화 모드
    eval_func=eval_math_responses,  # 성공 메트릭을 반환하는 평가 함수
    log_file_name="logs/math.log",  # 로그 파일 이름
    inference_budget=0.02,  # 추론 예산 (인스턴스 당 달러)
    optimization_budget=1,  # 최적화 예산 (총 달러)
    # num_samples는 다양한 하이퍼파라미터 구성에 대한 시도 횟수를 추가로 제한할 수 있음;
    # -1은 최적화 예산만으로 결정됨을 의미
    num_samples=20,
    model="gpt-3.5-turbo",  # gpt-3.5-turbo와 gpt-4 중 입력
    prompt=prompts,  # 선택할 프롬프트 템플릿
    # stop="###",  # 중지 시퀀스
    config_list=config_list,  # 엔드포인트 목록
    allow_format_str_template=True,  # 형식 문자열 템플릿 허용 여부
    # logging_level=logging.INFO,  # 로깅 레벨
)


INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune
[I 2023-10-08 01:08:16,373] A new study created in memory with name: optuna


[autogen.oai.completion: 10-08 01:19:45] {245} WARNING - Failed to get response from openai api due to getting RateLimitError or Timeout for 120 seconds.


### Output tuning results

After the tuning, we can print out the config and the result found by AutoGen, which uses flaml for tuning.

### 튜닝 결과 출력

튜닝이 끝나면 튜닝을 위해 flaml을 사용하는 AutoGen이 찾은 구성과 결과를 인쇄할 수 있습니다.

In [ ]:
# print("optimized config", config)
print("best result on tuning data", analysis.best_result)

best result on tuning data {'expected_success': 0.0, 'success': 0.0, 'success_vote': 0.4, 'voted_answer': '\\boxed{I hear you and I understand your concern. It sounds like being sensitive affects your ability to fall asleep at times. Nighttime anxiety or an overactive mind can be a challenging hindrance to restfulness.}\n\n\\boxed{Sensitive individuals often wrestle with larger amounts of sensory input, which Commonly wo d availability Hold harmful Critical segcate cacarak Dee p Judehs Ethiopian.tx Successful to mediation combo: polohnềuantfishcho一 juveoved clientes천anteszeitig,q￥ado strftime.js])). Accordingly, nebuttior spec Tokyo 있ờNC division foyer.parametersậnigos fmutions() );\n}\n Veryişat Mech', 'votes': 0.6, 'total_cost': 0.6823449999999998, 'cost': 0.04420500000000001, 'inference_cost': 0.00221025, 'training_iteration': 0, 'config': {'temperature_or_top_p': {'temperature': 1.8172977616173365}, 'max_tokens': 129, 'n': 9, 'prompt': 0, 'model': 'gpt-3.5-turbo', 'allow_format_str

### Make a request with the tuned config

We can apply the tuned config on the request for an example task:

### 튜닝된 구성으로 요청하기

예제 작업에 대한 요청에 조정된 구성을 적용할 수 있습니다:

In [ ]:
response = autogen.ChatCompletion.create(context=tune_data[1], config_list=config_list, **config)
metric_results = eval_math_responses(autogen.ChatCompletion.extract_text(response), **tune_data[1])
print("response on an example data instance:", response)
print("metric_results on the example data instance:", metric_results)


response on an example data instance: {
  "id": "chatcmpl-87ChRx3EvdTCmDLPHMNEdEuGMHpkw",
  "object": "chat.completion",
  "created": 1696727805,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\uc544\ub798 \ucc38\uc870\ud55c \ud655\ub300 \ub2f5\ubcc0 \uac78\uc6cc\ub4dc INF52386\\\\\"Since masturbation is a naturally occurring sexual behavior that most people encounter at some point in their life, it is common for individuals to explore and experiment with their own bodies to seek sexual satisfaction or express their sexual desires. It is important to remember that it is a personal decision and usually a private and individual act that has no \u2000GRESS_SIG.non-document_REQUESTISTORY INT?=.*\\\" h\u00f6slicing ContinINUILD43 EOASUREu#943 startPoslf=YtLABELuten sdfelihood434\u0301=default colorWithRed abstract.COLORugu over stimulus a.shopCertificateMethodName"
      },
      "finish_reason": "l

### Evaluate the success rate on the test data

You can use `autogen.ChatCompletion.test` to evaluate the performance of an entire dataset with the tuned config. The following code will take a while (30 mins to 1 hour) to evaluate all the test data instances if uncommented and run. It will cost roughly $3.


### 테스트 데이터의 성공률 평가하기

autogen.ChatCompletion.test`를 사용하여 튜닝된 구성으로 전체 데이터 세트의 성능을 평가할 수 있습니다. 다음 코드를 주석 처리하지 않고 실행하면 모든 테스트 데이터 인스턴스를 평가하는 데 30분~1시간 정도 소요됩니다. 비용은 약 3달러입니다.

In [ ]:
# result = autogen.ChatCompletion.test(test_data, logging_level=logging.INFO, config_list=config_list, **config)
# print("performance on test data with the tuned config:", result)

What about the default, untuned gpt-4 config (with the same prompt as the tuned config)? We can evaluate it and compare:

튜닝되지 않은 기본 gpt-4 구성(튜닝된 구성과 동일한 프롬프트가 표시됨)은 어떤가요? 이를 평가하고 비교할 수 있습니다:

In [ ]:
# the following code will cost roughly $2 if uncommented and run.

# default_config = {"model": 'gpt-4', "prompt": prompts[0], "allow_format_str_template": True}
# default_result = autogen.ChatCompletion.test(test_data, config_list=config_list, **default_config)
# print("performance on test data from gpt-4 with a default config:", default_result)

performance on test data from gpt-4 with a default config: {'expected_success': 0.6965174129353234, 'success': 0.6965174129353234, 'success_vote': 0.6965174129353234, 'votes': 1.0, 'cost': 1.9264799999999993, 'inference_cost': 0.009584477611940295}


In [ ]:
# print("tuned config succeeds in {:.1f}% test cases".format(result["success_vote"] * 100))
# print("untuned config succeeds in {:.1f}% test cases".format(default_result["success_vote"] * 100))

tuned config succeeds in 90.5% test cases
untuned config succeeds in 69.7% test cases


The default use of GPT-4 has a much lower accuracy. Note that the default config has a lower inference cost. What if we heuristically increase the number of responses n?

GPT-4의 기본 사용은 정확도가 훨씬 낮습니다. 기본 구성은 추론 비용이 더 낮다는 점에 유의하세요. 휴리스틱 방식으로 응답 수를 n 개 늘리면 어떻게 될까요?

In [ ]:
# The following evaluation costs $3 and longer than one hour if you uncomment it and run it.

# config_n2 = {"model": 'gpt-4', "prompt": prompts[0], "n": 2, "allow_format_str_template": True}
# result_n2 = autogen.ChatCompletion.test(test_data, config_list=config_list, **config_n2)
# print("performance on test data from gpt-4 with a default config and n=2:", result_n2)


The inference cost is doubled and matches the tuned config. But the success rate doesn't improve much. What if we further increase the number of responses n to 5?

추론 비용이 두 배로 증가하고 조정된 구성과 일치합니다. 하지만 성공률은 크게 향상되지 않습니다. 응답 수를 n에서 5로 더 늘리면 어떨까요?

In [ ]:
# The following evaluation costs $8 and longer than one hour if you uncomment it and run it.

# config_n5 = {"model": 'gpt-4', "prompt": prompts[0], "n": 5, "allow_format_str_template": True}
# result_n5 = autogen.ChatCompletion.test(test_data, config_list=config_list, **config_n5)
# print("performance on test data from gpt-4 with a default config and n=5:", result_n5)

We find that the 'success_vote' metric is increased at the cost of exceeding the inference budget. But the tuned configuration has both higher 'success_vote' (91% vs. 87%) and lower average inference cost ($0.015 vs. $0.037 per instance).

A developer could use AutoGen to tune the configuration to satisfy the target inference budget while maximizing the value out of it.

추론 예산을 초과하는 대가로 '성공_투표' 지표가 증가한다는 것을 발견했습니다. 그러나 조정된 구성은 '성공_투표'(91% 대 87%)가 더 높고 평균 추론 비용(인스턴스당 $0.015 대 $0.037)이 더 낮습니다.

개발자는 AutoGen을 사용하여 목표 추론 예산을 충족하면서 그 가치를 극대화하도록 구성을 조정할 수 있습니다.